# Transformer Models for Text

Setup and installation - we are going to use Transformers library from Huggingface

In [1]:
!wget https://raw.githubusercontent.com/huggingface/transformers/v4.5.1-release/examples/text-classification/run_glue.py -qq

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!pip install transformers datasets -qq
%env WANDB_DISABLE="true"


     |████████████████████████████████| 5.5 MB 30.6 MB/s 
     |████████████████████████████████| 451 kB 57.5 MB/s 
     |████████████████████████████████| 182 kB 65.5 MB/s 
     |████████████████████████████████| 7.6 MB 56.5 MB/s 
     |████████████████████████████████| 115 kB 64.7 MB/s 
     |████████████████████████████████| 212 kB 48.4 MB/s 
     |████████████████████████████████| 127 kB 65.1 MB/s 
env: WANDB_DISABLE="true"


Let's try our first Transformer model for text. We use run_glue.py script to fine-tune towards MRPC task (paraphrases). We train for 1 epoch with learning rate of 0 - so we're just using the downloaded model.


In [ ]:
%env TASK_NAME=MRPC

!python run_glue.py \
  --model_name_or_path bert-base-cased \
  --task_name $TASK_NAME \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 0 \
  --num_train_epochs 1 \
  --output_dir my_mrpc_init \
  --overwrite_output_dir

env: TASK_NAME=MRPC
INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
i

Does it work?

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tokenizer = AutoTokenizer.from_pretrained("my_mrpc_init")
model = AutoModelForSequenceClassification.from_pretrained("my_mrpc_init")
classes = ["not paraphrase", "is paraphrase"]
sequence_0 = "Tall mountains frequently have snow on top"
sequence_1 = "It's rare to have a beach at the top of the mountain"
sequence_2 = "Snow is often at the top of high mountains"
paraphrase = tokenizer(sequence_0, sequence_2, return_tensors="pt")
not_paraphrase = tokenizer(sequence_0, sequence_1, return_tensors="pt")
paraphrase_classification_logits = model(**paraphrase).logits
not_paraphrase_classification_logits = model(**not_paraphrase).logits
paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]
# Should be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")
# Should not be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")

ModuleNotFoundError: No module named 'transformers'

Not suprisingly, it does not work - the last layers that do the classification are randomly initialized. Let's train for 3 epochs with a small learning rate (2e-5)

Go to Runtime->Change Runtime Type and select GPU. Try also running it on CPU, and observe the difference in running time.


In [ ]:
%env TASK_NAME=MRPC

!python run_glue.py \
  --model_name_or_path bert-base-cased \
  --task_name $TASK_NAME \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 3 \
  --output_dir my_mrpc \
  --overwrite_output_dir

env: TASK_NAME=MRPC
INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
i

Now we can load the trained model again and see how well it works


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tokenizer = AutoTokenizer.from_pretrained("my_mrpc")
model = AutoModelForSequenceClassification.from_pretrained("my_mrpc")
classes = ["not paraphrase", "is paraphrase"]
sequence_0 = "Tall mountains frequently have snow on top"
sequence_1 = "It's rare to have a beach at the top of the mountain"
sequence_2 = "Snow is often at the top of high mountains"
paraphrase = tokenizer(sequence_0, sequence_2, return_tensors="pt")
not_paraphrase = tokenizer(sequence_0, sequence_1, return_tensors="pt")
paraphrase_classification_logits = model(**paraphrase).logits
not_paraphrase_classification_logits = model(**not_paraphrase).logits
paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]
# Should be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")
# Should not be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")

not paraphrase: 13%
is paraphrase: 87%
not paraphrase: 80%
is paraphrase: 20%


Works!

For standard tasks, we don't have to train outselves, there are pre-trained, fine-tuned models ready for download

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")
classes = ["not paraphrase", "is paraphrase"]
sequence_0 = "Tall mountains frequently have snow on top"
sequence_1 = "It's rare to have a beach at the top of the mountain"
sequence_2 = "Snow is often at the top of high mountains"
paraphrase = tokenizer(sequence_0, sequence_2, return_tensors="pt")
not_paraphrase = tokenizer(sequence_0, sequence_1, return_tensors="pt")
paraphrase_classification_logits = model(**paraphrase).logits
not_paraphrase_classification_logits = model(**not_paraphrase).logits
paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]
# Should be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")
# Should not be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

not paraphrase: 6%
is paraphrase: 94%
not paraphrase: 95%
is paraphrase: 5%


Let's see what is the model architecture. We can just do print(model). Recall the extra_repr() module function that allows any module to print meaningful info about itself.

We'll see that the model uses the standard design elements we've discussed: LayerNorm, Dropout (and skip connections, we don't see those here, but they're in the code).

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

Let's try a different task: sentiment analysis. We'll use a pre-trained, fine-tuned model again. 

In [ ]:
from transformers import pipeline
nlp = pipeline("sentiment-analysis")
result = nlp("The huggingface library is quite comprehensive and simple to use")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")
result = nlp("Coding it all in pytorch would be a daunting task")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

label: POSITIVE, with score: 0.9993
label: NEGATIVE, with score: 0.993


Note that a "distillBERT" model was used: a maller model resultng knowledge distillation applied to a larger BERT model.

Transformers can also be used to generate text. For example, to create summaries. Observe the size of the downloaded model.

In [ ]:
from transformers import pipeline
import textwrap

to_tokenize = "The idea that aliens had frequented our planet had been circulating among ufologists since the postwar years, when a Polish emigre, George Adamski, claimed to have rendezvoused with a race of kindly, Nordic-looking Venusians who were disturbed by the domestic and interplanetary effects of nuclear-bomb tests. In the summer of 1947, an alien spaceship was said to have crashed near Roswell, New Mexico. Conspiracy theorists believed that vaguely anthropomorphic bodies had been recovered there, and that the crash debris had been entrusted to private military contractors, who raced to unlock alien hardware before the Russians could. (Documents unearthed after the fall of the Soviet Union suggested that the anxiety about an arms race supercharged by alien technology was mutual.) All of this, ufologists claimed, had been covered up by Majestic 12, a clandestine, para-governmental organization convened under executive order by President Truman. President Kennedy was assassinated because he planned to level with Premier Khrushchev; Kennedy had confided in Marilyn Monroe, thereby sealing her fate. Representative Steven Schiff, of New Mexico, spent years trying to get to the bottom of the Roswell incident, only to die of 'cancer'."

# Initialize the HuggingFace summarization pipeline
summarizer = pipeline("summarization",model="t5-base", tokenizer="t5-base")
summarized = summarizer(to_tokenize, min_length=40, max_length=150)



Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
# Print original text
print(textwrap.fill(to_tokenize, 80))


The idea that aliens had frequented our planet had been circulating among
ufologists since the postwar years, when a Polish emigre, George Adamski,
claimed to have rendezvoused with a race of kindly, Nordic-looking Venusians who
were disturbed by the domestic and interplanetary effects of nuclear-bomb tests.
In the summer of 1947, an alien spaceship was said to have crashed near Roswell,
New Mexico. Conspiracy theorists believed that vaguely anthropomorphic bodies
had been recovered there, and that the crash debris had been entrusted to
private military contractors, who raced to unlock alien hardware before the
Russians could. (Documents unearthed after the fall of the Soviet Union
suggested that the anxiety about an arms race supercharged by alien technology
was mutual.) All of this, ufologists claimed, had been covered up by Majestic
12, a clandestine, para-governmental organization convened under executive order
by President Truman. President Kennedy was assassinated because he plan

In [ ]:
# Print summarized text
summary=summarized[0]['summary_text']
print(textwrap.fill(summary, 80))

a spaceship crashed in 1947 near Roswell, new mexico . ufologists claimed that
private contractors had entrusted the debris to private contractors . the debris
had been used to unlock alien hardware before the Russians could .


Let's look at the code that actually uses the model (T5) directly.

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer

to_tokenize = "The idea that aliens had frequented our planet had been circulating among ufologists since the postwar years, when a Polish emigre, George Adamski, claimed to have rendezvoused with a race of kindly, Nordic-looking Venusians who were disturbed by the domestic and interplanetary effects of nuclear-bomb tests. In the summer of 1947, an alien spaceship was said to have crashed near Roswell, New Mexico. Conspiracy theorists believed that vaguely anthropomorphic bodies had been recovered there, and that the crash debris had been entrusted to private military contractors, who raced to unlock alien hardware before the Russians could. (Documents unearthed after the fall of the Soviet Union suggested that the anxiety about an arms race supercharged by alien technology was mutual.) All of this, ufologists claimed, had been covered up by Majestic 12, a clandestine, para-governmental organization convened under executive order by President Truman. President Kennedy was assassinated because he planned to level with Premier Khrushchev; Kennedy had confided in Marilyn Monroe, thereby sealing her fate. Representative Steven Schiff, of New Mexico, spent years trying to get to the bottom of the Roswell incident, only to die of 'cancer'."

model = AutoModelWithLMHead.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")
# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer.encode("summarize: " + to_tokenize, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1136: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
print(inputs)

tensor([[21603,    10,    37,   800,    24, 12430,     7,   141,  8325,    15,
            26,    69,  4345,   141,   118,     3, 30715,   859,     3,    76,
            89, 17382,   437,     8,   442,  2910,   203,     6,   116,     3,
             9, 16073,     3,    15, 10673,    60,     6,  3080,  7124,  4009,
             6,  7760,    12,    43, 14380,  3249,    15,    26,    28,     3,
             9,  1964,    13,     3, 21914,     6, 24207,    18, 10119, 22301,
          7137,   113,   130, 30371,    57,     8,  4422,    11,  1413, 30351,
          1951,    13,  6414,    18,   115,  8038,  3830,     5,    86,     8,
          1248,    13, 23992,     6,    46, 12430,   628,  2009,    47,   243,
            12,    43, 24679,  1084,  7963,  2091,     6,   368,  4726,     5,
          7762,  2388,  4710,     8,   127,   343,     7,  6141,    24, 15986,
           120,     3, 21468,  8886,   447,  5678,   141,   118, 16599,   132,
             6,    11,    24,     8,  8420, 12544,  

In [ ]:
print(outputs)

tensor([[    0,     3,     9,   628,  2009, 24679,    16, 23992,  1084,  7963,
          2091,     6,   126,   140,   226,  5807,     3,     5,     3,    76,
            89, 17382,  7760,    24,  1045,  8392,   141,     3,    35,  2666,
          6265,     8, 12544,    12,  1045,  8392,     3,     5,     8, 12544,
           141,   118,     3,    35,  2666,  6265,    12,  1045,  8392,     6,
             3,    76,    89, 17382,  7760,     3,     5,     1]])


In [ ]:
print(tokenizer.decode(outputs[0]))

<pad> a spaceship crashed in 1947 near Roswell, new mexico. ufologists claimed that private contractors had entrusted the debris to private contractors. the debris had been entrusted to private contractors, ufologists claimed.</s>


Let's also try translation

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

to_tokenize = "The idea that aliens had frequented our planet had been circulating among ufologists since the postwar years, when a Polish emigre, George Adamski, claimed to have rendezvoused with a race of kindly, Nordic-looking Venusians who were disturbed by the domestic and interplanetary effects of nuclear-bomb tests. In the summer of 1947, an alien spaceship was said to have crashed near Roswell, New Mexico. Conspiracy theorists believed that vaguely anthropomorphic bodies had been recovered there, and that the crash debris had been entrusted to private military contractors, who raced to unlock alien hardware before the Russians could. (Documents unearthed after the fall of the Soviet Union suggested that the anxiety about an arms race supercharged by alien technology was mutual.) All of this, ufologists claimed, had been covered up by Majestic 12, a clandestine, para-governmental organization convened under executive order by President Truman. President Kennedy was assassinated because he planned to level with Premier Khrushchev; Kennedy had confided in Marilyn Monroe, thereby sealing her fate. Representative Steven Schiff, of New Mexico, spent years trying to get to the bottom of the Roswell incident, only to die of 'cancer'."

model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")
# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer.encode("translate English to German: " + to_tokenize, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(inputs, max_length=512, min_length=40, num_beams=4, early_stopping=True)
print(tokenizer.decode(outputs[0]))

<pad> Die Idee, dass Außerirdische unseren Planeten bewohnten, kursierte unter Ufologen seit den Nachkriegsjahren, als ein polnischer Emigrant, George Adamski, behauptete, mit einer Rasse von freundlichen, nordischen Venusiern zusammengetroffen zu sein, die durch die heimischen und interplanetären Auswirkungen von Atombombentests gestört waren. Im Sommer 1947 soll ein Außerirdische</s>
